In [2]:
#!/usr/bin/env python
"""Script to compare several variant calling csv file 
in the frame of Performance testing of plant virus 
(followong COST-DIVAS action)."""

import argparse
import os
import pandas as pd
import statistics
import numpy as np
import matplotlib.pyplot as plt

def make_percentage(freq):
    """
    """
    if "%" in freq:
        freq = float(freq.split("%")[0])
        freq = freq/100
    return float(freq)
    
def clean_data(new_df, clean_virus_data):
    """
    """
    col_name = ["Lab", "File_name", "Input_file", "Org_ref", "Position", \
        "Type", "Reference","Allele", "Coverage", "Average_quality", \
        "Variant_frequency", "Personal_validation", "Comment"]
    clean_virus_tmp = pd.DataFrame(columns=col_name)
    clean_virus_list = []
    
    # print(new_df.loc[new_df.index[pd.isna(new_df["File name"])]])
    dumb_list1 = []
    dumb_list2 = []
    dumb_list3 = []

    # remove row when "File name" is null
    update_df = new_df.drop(new_df.index[pd.isna(new_df["File name"])])

    for index, row in update_df.iterrows():
        ######## file name column
        el = row["File name"]
        data = el.split("_")
        input_file_data = str(data[0] + "_" + data[1])
        org_ref_data = data[-1]
        if "." in org_ref_data:
            org_ref_data = org_ref_data.split(".")[0]
        if " " in org_ref_data:
            org_ref_data = org_ref_data.split(" ")[0]
        file_name = str(input_file_data + "_" + org_ref_data )
        update_df.loc[index,"Input_file"] = input_file_data
        update_df.loc[index,"Org_ref"] = org_ref_data
        update_df.loc[index,"File_name"] = file_name
        ########
        ######## Position column
        el = str(row["Position"])
        if ".." in el:
            pos1 = int(float(el.split("..")[0]))
            pos2 = int(float(el.split("..")[1]))
        elif "^" in el:
            pos1 = int(float(el.split("^")[0]))
            pos2 = int(float(el.split("^")[1]))      
        elif pd.isna(row["Position"]):
            pos1 = None
            pos2 = None
        else:
            pos1 = int(float(el))
            pos2 = None
        update_df.loc[index,"Pos_1"] = pos1
        update_df.loc[index,"Pos_2"] = pos2
        ########
        ######## Coverage column       
        el = str(row["Coverage"])
        if "->" in el:
            cov1 = int(float(el.split("->")[0]))
            cov2 = int(float(el.split("->")[1]))
        elif "^" in el:
            cov1 = int(float(el.split("^")[0]))
            cov2 = int(float(el.split("^")[1]))
        elif ".." in el:
            cov1 = int(float(el.split("..")[0]))
            cov2 = int(float(el.split("..")[1]))            
        elif pd.isna(row["Coverage"]):
            cov1 = None
            cov2 = None
        else:
            cov1 = int(float(el))
            cov2 = None
        update_df.loc[index,"Cov_1"] = cov1
        update_df.loc[index,"Cov_2"] = cov2
        ########
        ######## Variant frequency  
        lab_percent_list = ["B", "H", "O", "Q", "Y"]
        el = str(row["Variant Frequency"])
        if "," in el:
            el = el.replace(",", ".")
        if "->" in el:
            freq1 = el.split("->")[0]
            freq1 = make_percentage(freq1)
            freq2 = el.split("->")[1]
            freq2 = make_percentage(freq2)
        elif "^" in el:
            freq1 = el.split("^")[0]
            freq1 = make_percentage(freq1)
            freq2 = el.split("^")[1]
            freq2 = make_percentage(freq2)
        elif ".." in el:
            freq1 = el.split("..")[0]
            freq1 = make_percentage(freq1)
            freq2 = el.split("..")[1]   
            freq2 = make_percentage(freq2)
        elif pd.isna(row["Variant Frequency"]):
            freq1 = None
            freq2 = None
        else:
            freq1 = el
            freq1 = make_percentage(freq1)
            freq2 = None
            
        if str(row["Lab"]) in lab_percent_list:
            if freq1 != None:
                freq1 = float(freq1)/100
            if freq2 != None:
                freq2 = float(freq2)/100
            
        update_df.loc[index,"frequency_1"] = freq1
        update_df.loc[index,"frequency_2"] = freq2        
        
        dumb_list1.append(el)
        dumb_list2.append(input_file_data)
        dumb_list3.append(org_ref_data)
    
    #update_df = update_df.drop(["File name"], axis=1)
    # print("File name")
    # print(set(dumb_list1))  
    # print("input_file_data")
    # print(set(dumb_list2))   
    # print("org_ref_data")
    # print(set(dumb_list3))    
    clean_virus_data = clean_virus_data.append(update_df, ignore_index=True)
    # print(clean_virus_data)

    return clean_virus_data

def open_file(full_path, col_name, lab_dir, clean_virus_data):
    """
    """
    # print(full_path)
    xls = pd.ExcelFile(full_path, engine='openpyxl')
    sheet_list = xls.sheet_names
    for sheet in sheet_list:
        # print(sheet)
        virus_data = pd.read_excel(full_path, engine='openpyxl', sheet_name=sheet)
        virus_data.insert(0,"Lab", lab_dir)
        selected_columns = virus_data[col_name]
        new_df = selected_columns.copy()
        clean_virus_data = clean_data(new_df, clean_virus_data)
    return clean_virus_data

if __name__ == "__main__":

    #TODO argparse use argument ?
    col_name = ["Lab", "File name", "Position", \
        "Type", "Reference","Allele", "Coverage", "Average quality", \
        "Variant Frequency", "Personal validation", "Comment"]

    out_dir = "/mnt/c/Users/johan/OneDrive/Bureau/bioinfo/PT3/Result/variant_calling/"
    lab_dir = out_dir
    clean_virus_data = pd.DataFrame(columns=col_name)
    
    for lab_dir in os.listdir(out_dir):
        #print(lab_dir)
        
        filename_list = os.listdir(os.path.join(out_dir, lab_dir))
        #filename_list = os.listdir(out_dir)
        for filename in filename_list:
            if filename.startswith("PT3_snp_template"):
                print(filename)
                full_path = os.path.join(os.path.join(out_dir, lab_dir),filename)
                # full_path = os.path.join(out_dir,filename)
                clean_virus_data = open_file(full_path, col_name, lab_dir, clean_virus_data)
        
        clean_virus_data = clean_virus_data.drop(["File name"], axis=1)
        clean_virus_data = clean_virus_data.drop(["Position"], axis=1)
        clean_virus_data = clean_virus_data.drop(["Coverage"], axis=1)
        clean_virus_data = clean_virus_data.drop(["Variant Frequency"], axis=1)
        
    #print(clean_virus_data)
    


PT3_snp_template_A.xlsx
PT3_snp_template_lab_B.xlsx
PT3_snp_template_C.xlsx
PT3_snp_template_D.xlsx
PT3_snp_template_labG.xlsx
PT3_snp_template_H.xlsx
PT3_snp_template_O.xlsx
PT3_snp_template_Lab P.xlsx
PT3_snp_templateQdraft_edited_ae_E.xlsx
PT3_snp_templateQdraft_edited_ae_F.xlsx
PT3_snp_templateQdraft_edited_ae_G.xlsx
PT3_snp_templateQdraft_edited_ae_Q.xlsx
PT3_snp_template_R.xlsx
PT3_snp_template_filled_labS.xlsx
PT3_snp_template_Lab T.xlsx
PT3_snp_template_Y.xlsx
PT3_snp_template_Z.xlsx


In [2]:

#rint(clean_virus_data.groupby("Lab")['frequency_1'].max())

#print(clean_virus_data["frequency_1"].unique())
#print(len(clean_virus_data["frequency_1"].unique()))

In [3]:
count=0
max_nb = float(0)
min_nb = float(1)
for index, element in clean_virus_data.iterrows():
    if not pd.isna(element["frequency_1"]):
        a = element["frequency_1"]
        if max_nb < a:
            max_nb = a
        if min_nb > a:
            min_nb = a
                

        
        #if element["Lab"] == ">=":
            #print(element["Lab"])
            #print(index)
            #print(element.loc[index, "File_name"])
            #count+=1
            
            #print(clean_virus_data.iloc[[str(int(index-1))]])
        
print(max_nb)
print(min_nb)

1.0
0.001


In [4]:
#print(clean_virus_data.File_name.unique())
print(len(clean_virus_data.File_name.unique()))


125


In [5]:
clean_virus_data.head()

,Lab,Type,Reference,Allele,Average quality,Personal validation,Comment,Input_file,Org_ref,File_name,Pos_1,Pos_2,Cov_1,Cov_2,frequency_1,frequency_2
0,A,Deletion,G,NaN,36,Yes,NaN,dataset_1BA,DQ000985,dataset_1BA_DQ000985,319.0,NaN,5339.0,NaN,0.196,NaN
1,A,Deletion,G,NaN,36,Yes,NaN,dataset_1CA,DQ000985,dataset_1CA_DQ000985,319.0,NaN,5783.0,NaN,0.189,NaN
2,A,Deletion,G,NaN,36,Yes,NaN,dataset_1DA,DQ000985,dataset_1DA_DQ000985,319.0,NaN,5783.0,NaN,0.189,NaN
3,A,Deletion,G,NaN,36,Yes,NaN,dataset_1BE,DQ000985,dataset_1BE_DQ000985,319.0,NaN,5268.0,NaN,0.198,NaN
4,A,Deletion,G,NaN,35,Yes,NaN,dataset_1CE,DQ000985,dataset_1CE_DQ000985,319.0,NaN,5746.0,NaN,0.191,NaN


In [4]:
import qgrid

A = qgrid.show_grid(clean_virus_data)
A

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [5]:
print(clean_virus_data.Reference.unique())
#clean_virus_data.to_csv("/mnt/c/Users/johan/OneDrive/Bureau/test_PT3.csv", index=False, sep=';', encoding='utf-8')

['G' 'A' nan 'C' 'T' 'AC' 'TG' 'CT' 'CAAC' 'W' '-' 'AA' '/' 'ACT' 'TC'
 'GC' 'TT' 'CA' 'CG' 'AT' 'GAG' 'CAA' 'AAG' 'TA' 'CCG' 'CTT' 'CC' 'GT'
 'GA' 'ACAC' 'GGT' 'AAA' 'AGC' 'CAAA' 'GG' 'CCC' 'GTGT' 'AGT' 'ATG' 'AG'
 'CGT' 'U' 'GCT']
